In [14]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
import pandas as pd  # Add pandas to handle Excel files

path = 'images'
images = []
classNames = []

# Read images and class names
myList = os.listdir(path)
print(myList)
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)  # Append the image here
    classNames.append(os.path.splitext(cl)[0])  # Append the class name here

print(classNames)

def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encodes = face_recognition.face_encodings(img)
        if len(encodes) > 0:  # Check if at least one face is found
            encode = encodes[0]
            encodeList.append(encode)
    return encodeList

def markAttendance(name):
    # Use pandas to read/write to the Excel file
    try:
        # Try reading the Excel file
        df = pd.read_excel('attendance_system.xlsx')
    except FileNotFoundError:
        # If file doesn't exist, create an empty DataFrame with 'Name' and 'Time' columns
        df = pd.DataFrame(columns=['Name', 'Time'])

    # Ensure 'Name' column exists in case the Excel file structure is incorrect
    if 'Name' not in df.columns:
        df['Name'] = []

    # Check if the name is already in the attendance list
    if name not in df['Name'].values:
        now = datetime.now()
        dtString = now.strftime('%H:%M:%S')
        # Append the new record to the DataFrame
        new_row = pd.DataFrame({'Name': [name], 'Time': [dtString]})
        df = pd.concat([df, new_row], ignore_index=True)
        # Save the updated DataFrame back to the Excel file
        df.to_excel('attendance_system.xlsx', index=False)


# Encode the known faces
encodeListKnown = findEncodings(images)
print('Encoding Complete')

cap = cv2.VideoCapture(0)
frame_resizing = 0.25

while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0, 0), fx=frame_resizing, fy=frame_resizing)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
 
    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)
 
    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        print('matches', matches)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        print(faceDis)
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            print(name)
            
            faceLoc = np.array(faceLoc) / frame_resizing  # Resize face location back
            faceLoc = faceLoc.astype(int)
            y1, x2, y2, x1 = faceLoc  # Unpack face location
         
            # Draw rectangle around the face
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            
            # Mark attendance
            markAttendance(name)
 
    cv2.imshow('Webcam', img)
    key = cv2.waitKey(1)
    if key == 27:  # Press ESC to break the loop
        break

cap.release()
cv2.destroyAllWindows()


['amr-sanad.jpg', 'amr.jpeg', 'kareem.jpeg', 'khaled.jpeg', 'mero.jpeg', 'sanad.jpeg']
['amr-sanad', 'amr', 'kareem', 'khaled', 'mero', 'sanad']
Encoding Complete
matches [True, False, False, False, False, False]
[0.59399707 0.75212428 0.73850585 0.89472788 0.86916294 0.76755242]
AMR-SANAD
matches [False, False, False, False, False, False]
[0.62226492 0.66984191 0.62440475 0.87813825 0.8473018  0.6242901 ]
matches [True, False, False, False, False, False]
[0.45441413 0.67692667 0.68784039 0.85998613 0.81892579 0.7172629 ]
AMR-SANAD
matches [True, False, False, False, False, False]
[0.44740733 0.69267139 0.72988052 0.86249239 0.93356787 0.71718974]
AMR-SANAD
matches [True, False, False, False, False, False]
[0.48922319 0.73502217 0.68903852 0.90715131 0.90896192 0.72418613]
AMR-SANAD
matches [True, False, False, False, False, False]
[0.44826034 0.72873634 0.72114772 0.86025847 0.91404031 0.7263193 ]
AMR-SANAD
matches [True, False, False, False, False, False]
[0.54954326 0.80134959 0.763

In [12]:
!pip install openpyxl


   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.9 kB ? eta -:--:--
   ------ -------------------------------- 41.0/250.9 kB 487.6 kB/s eta 0:00:01
   ----------------- -------------------- 112.6/250.9 kB 930.9 kB/s eta 0:00:01
   ----------------------------------- ---- 225.3/250.9 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 1.4 MB/s eta 0:00:00


DEPRECATION: vtk -PKG-VERSION has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of vtk or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
